In [1]:
import os

import schematics as scm
import schematics.types as scmt

import pandas as pd
import numpy as np

%cd ~/flatiron/python

from flatiron.core.pipeline import *

/home/ubuntu/flatiron/python


2023-01-31 09:01:30.933576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class FooConfig(scm.Model):
    bar = scmt.StringType(required=True)

class FooPipeline(PipelineBase):
    _model_config_class = FooConfig
    
    def build(self):
        pass

def create_fake_asset(root):
    chunks = [
        'data/chunk_f00.npy',
        'data/chunk_f01.npy',
    ]
    os.makedirs(root, exist_ok=True)
    os.makedirs(Path(root, 'data'), exist_ok=True)
    data = pd.DataFrame()
    data['filepath_relative'] = chunks
    data['asset_path'] = root
    csv_path = Path(root, 'info.csv').as_posix()
    data.to_csv(csv_path, index=False)
    shape = (100, 10, 10, 3)
    arr = np.ones(shape, dtype=np.float16)
    for chunk in chunks:
        np.save(Path(root, chunk), arr)
    return data

In [5]:
root = '/tmp/foobar'
d = create_fake_asset(root)
config = dict(
    model=dict(bar='baz'),
    dataset=dict(
        source=root,
        # load_limit=100,
        split_index=3,
    )
)
p = FooPipeline(config) \
    .load() \
    .train_test_split() \
    .unload()

p.x_train.shape, p.x_test.shape

Loading Dataset Chunks:   0%|          | 0/2 [00:00<?, ?it/s]

((160, 10, 10, 3), (40, 10, 10, 3))